In [1]:
from surprise import KNNBaseline, KNNWithMeans, KNNBasic, BaselineOnly, KNNWithZScore
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from surprise import SVDpp

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

%matplotlib inline

In [2]:
dict_for_grades = {'vl-0': 0, 'vl-1': 1, 'vl-2': 2, 'vl-3': 2, 'vl-4': 3, 'vl-5': 3, 'vl-6': 4, 'vl-7': 4, 'vl-8': 4, 'vl-9': 5, 'vl-10': '5a', 'vl-11': '5a+', 'vl-12': '5b', 'vl-13': '5b+', 'vl-14': '5c', 'vl-15': '5c+', 'vl-16': '6a', 'vl-17': '6a+', 'vl-18': '6b', 'vl-19': '6b+', 'vl-20': '6b+', 'vl-21': '6c', 'vl-22': '6c+', 'vl-23': '7a', 'vl-24': '7a+', 'vl-25': '7b', 'vl-26': '7b+', 'vl-27': '7c', 'vl-28': '7c+', 'vl-29': '8a', 'vl-30': '8a+', 'vl-31': '8b', 'vl-32': '8b+', 'vl-33': '8c', 'vl-34': '8c+', 'vl-35': '9a', 'vl-36': '9a+', 'vl-37': '9b', 'vl-38': '9b+', 'vl-39': '9c'}

# Surprise library implementation

In [3]:
train_file_name = 'data/train_ds_0_2.csv'
#'data/TRAIN_routes_v1.csv'

#'data/train_ds_0_2.csv' 
#'data/train_ds_ds_v7_allhist.csv'
#'data/train_ds_ds_v7_single_time_all_hist.csv'
test_file_name = 'data/test_ds_0_2.csv'
#'data/TEST_routes_v1.csv'
#'data/test_ds_0_4_rand_42.csv'
#'data/test_ds_0_2.csv'
#'data/test_ds_ds_v7_allhist.csv'
#'data/test_ds_ds_v7_single_time_all_hist.csv'
df_train_svd = pd.read_csv(train_file_name)
df_test_svd = pd.read_csv(test_file_name)

In [4]:
df_train_svd['diff'] = df_train_svd['user_grade_id'] - df_train_svd['grade_id'] + 4
df_test_svd['diff'] = df_test_svd['user_grade_id'] - df_test_svd['grade_id'] + 4

In [6]:
# A reader is still needed but only the rating_scale param is requiered.
mmin, mmax = 1, 7
reader = Reader(rating_scale=(mmin, mmax))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_train_svd[['user_id','route_id','diff']], reader)

In [7]:
# testset_model_surprise = Dataset.load_from_df(df_test_svd[['user_id','route_id','diff']], reader).build_full_trainset()
trainset_model_surprise = Dataset.load_from_df(df_train_svd[['user_id','route_id','diff']], reader)
# testset = testset_model_surprise.build_testset()
testset = df_test_svd[['user_id','route_id','diff']].values
trainset = trainset_model_surprise.build_full_trainset()

In [388]:
print('Using ALS')
bsl_options_als = [

    {
        'method': 'als',
        'n_epochs': 100,
        'reg_u': 5,
        'reg_i': 3
    }
]
for bsl_options_al in bsl_options_als:
    print('reg_i: ', bsl_options_al['reg_i'])
    algo = BaselineOnly(bsl_options=bsl_options_al)
    predictions = algo.fit(trainset).test(testset)

    accuracy.rmse(predictions)
    accuracy.mse(predictions)
    accuracy.mae(predictions)

Using ALS
reg_i:  3
Estimating biases using als...
RMSE: 1.2649
MSE: 1.5999
MAE:  1.1252


In [155]:
#df_test_svd[['user_id', 'route_id', 'SVD_prediction']].to_csv('data/test_ds_0_2_SVD_predictions.csv', index=False)
#df_test_svd[['user_id', 'route_id', 'SVD_prediction']].to_csv('data/test_ds_0_4_rand_42_SVD_predictions.csv', index=False)
df_test_svd[['user_id', 'route_id', 'SVD_prediction']].to_csv('data/TEST_routes_v1_SVD_predictions.csv', index=False)

In [ ]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_train_svd[['user_id','route_id','diff']], reader)

In [ ]:
param_grid = {'n_epochs': [20, 30, 50],
              'n_factors': [20, 30, 40, 100],
              'biased': [True],
              'lr_bu': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'lr_bi': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'lr_pu': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'lr_qi': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'reg_bu': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'reg_bi': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'reg_pu': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
              'reg_qi': [0.5, 0.1, 0.05, 0.005, 0.001, 0.0001],
             }
                                               
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

After finding the suitable parameters made via cross validation, paste them and run trhe code:

In [241]:
list_lr = [3, 0.5, 0.1, 0.05, 0.005, 0.001, 0.0001]
list_reg = [0.75, 0.5, 0.1, 0.05, 0.036, 0.001, 0.0001]
list_lr_bu = [0.001]
list_lr_bi = [0.001]
list_lr_pu = [0.1]
list_lr_qi = [0.1]
list_reg_bu = [0.5]
list_reg_bi = [0.5]
list_reg_pu = [0.1]
list_reg_qi = [0.05]
list_epochs = [40]
for epochs_ in list_epochs:
    for lr_bu_ in list_lr_bu:
        for lr_bi_ in list_lr_bi:
            for lr_pu_ in list_lr_pu:
                for lr_qi_ in list_lr_qi:
                    for reg_bu_ in list_reg_bu:
                        for reg_bi_ in list_reg_bi:
                            for reg_pu_ in list_reg_pu:
                                for reg_qi_ in list_reg_qi:
                                    algo = SVD(n_factors=40, # The number of factors. Default is 100.
                                               n_epochs=epochs_, # The number of iteration of the SGD procedure. Default is 20.
                                               biased = True, # Whether to use baselines (or biases). See note above. Default is True.
                                               init_mean =0, # The mean of the normal distribution for factor vectors initialization. Default is 0.
                                               init_std_dev=0.1, # The standard deviation of the normal distribution for factor vectors initialization.
                                                                   # Default is 0.1.
                                               lr_all=0.005, # The learning rate for all parameters. Default is 0.005.
                                               reg_all=0.1, # The regularization term for all parameters. Default is 0.02.
                                               lr_bu=lr_bu_, # The learning rate for 𝑏𝑢.
                                               lr_bi=lr_bi_, #
                                               lr_pu=lr_pu_, #
                                               lr_qi=lr_qi_, #
                                               reg_bu=reg_bu_, #
                                               reg_bi=reg_bi_, #
                                               reg_pu=reg_pu_, #
                                               reg_qi=reg_qi_ #
                                              )
                                    algo.fit(trainset)
                                    # Evaluation
                                    predictions = algo.test(testset)
                                    print(' epochs: ', epochs_, ' lr_bu: ', lr_bu_, ' lr_bi: ', lr_bi_, ' lr_pu: ', lr_pu_, ' lr_qi: ', lr_qi_)
                                    print('reg_bu: ', reg_bu_, ' reg_bi: ', reg_bi_, ' reg_pu: ', reg_pu_, ' reg_qi: ', reg_qi_)
                                    print(f'MAE: {accuracy.mae(predictions, verbose=False):.3f}')
                                    print(f'MSE : {accuracy.mse(predictions, verbose=False):.3f}')
                                    print(f'RMSE: {accuracy.rmse(predictions, verbose=False):.3f}')

 epochs:  40  lr_bu:  0.001  lr_bi:  0.001  lr_pu:  0.1  lr_qi:  0.1
reg_bu:  0.5  reg_bi:  0.5  reg_pu:  0.1  reg_qi:  0.05
MAE: 0.102
MSE : 0.095
RMSE: 0.307
